In [74]:
from bibutils import load_bib, write_bib
from titlecase import titlecase
import calendar
import re

BIB_FILENAME = 'mendeley-processed.bib'
#BIB_FILENAME = 'library.bib'

bib_database = load_bib(filename=BIB_FILENAME)

In [78]:
len(bib_database.entries)

256

In [76]:
def change_journal_to_journaltitle(entry):
    journal = entry.pop('journal', None)
    if journal:
        print(u"Changing 'journal={}' to 'journaltitle' from ID:{}".format(journal, entry['ID']))
        entry['journaltitle'] = journal

def remove_curly_brackets(string):
    return string.replace('{', '').replace('}', '')

def change_month(entry):
    try:
        month = entry['month'].lower()
    except KeyError:
        return
    
    def replace_month_name_with_int(month_array):
        month_names = [month_name.lower() for month_name in month_array]
        if month in month_names:
            month_i = str(month_names.index(month))
            print(u"Changing month from", month, u"to", month_i)
            entry['month'] = month_i
    replace_month_name_with_int(calendar.month_name)
    replace_month_name_with_int(calendar.month_abbr)
    
def lower_case_id(entry):
    bib_id = entry['ID']
    if len(bib_id) >= 2 and bib_id[1].islower():
        entry['ID'] = bib_id[0].lower() + bib_id[1:]

In [77]:
for entry in bib_database.entries:
    change_journal_to_journaltitle(entry)
    entry['title'] = remove_curly_brackets(entry['title'])
    entry['title'] = titlecase(entry['title'])
    try:
        entry['author'] = remove_curly_brackets(entry['author'])
    except KeyError:
        pass
    for key_to_delete in ['keywords', 'file', 'mendeley-tags']:
        entry.pop(key_to_delete, None)
    change_month(entry)
    entry.pop('mendeley-tags', '')
    lower_case_id(entry)
    
    # Set 'publisher'
    for publisher in ['ACM', 'IEEE']:
        if ('publisher' not in entry) and (publisher in entry['title'] or publisher in entry.get('booktitle', '')):
            entry['publisher'] = publisher
            print("setting publisher to", publisher, "for ID", entry['ID'])
            
    # Remove 'year' and 'month' and use 'date' instead
    if 'date' in entry:
        if 'year' in entry:
            print("Removing year from", entry['ID'])
            if entry['year'] != entry['date'][:4]:
                print('*****************', entry['year'], entry['date'], entry['ID'])
                #raise RuntimeError()
            entry.pop('year')
        if 'month' in entry:
            print("Removing month from", entry['ID'])
            if '{:02d}'.format(int(entry['month'])) != entry['date'][5:7]:
                print('*************month', '{:02d}'.format(int(entry['month'])), entry['date'], entry['ID'], entry['date'][5:7])
                #raise RuntimeError()
            entry.pop('month')
    else:
        if 'year' in entry:
            entry['date'] = entry.pop('year')
            if 'month' in entry:
                entry['date'] += '-' + '{:02d}'.format(int(entry.pop('month'))) 
            print("Setting date to", entry['date'], "for", entry['ID'])

    if 'date' in entry:
        if not re.match(r"""\d{4}-\d{2}-\d{2}""", entry['date']):
            if not re.match(r"""\d{4}-\d{2}""", entry['date']):
                if not re.match(r"""\d{4}""", entry['date']):
                    print("********##### BAD DATE", entry['date'], entry['ID'])

Changing month from aug to 8
Setting date to 2007-08 for 
Setting date to 2007 for 
Setting date to 2008 for 
Changing month from oct to 10
Setting date to 2008-10 for 
Setting date to 2009 for 
Changing 'journal=Power Delivery, IEEE Transactions on' to 'journaltitle' from ID:
Changing month from apr to 4
Setting date to 2010-04 for 
Changing month from nov to 11
Setting date to 2010-11 for 
Setting date to 2010 for 
Changing month from sep to 9
Setting date to 2010-09 for 
Changing month from aug to 8
Setting date to 2011-08 for 
Setting date to 2011 for 
Changing month from apr to 4
Setting date to 2011-04 for 
Setting date to 2011 for 
Changing month from sep to 9
Setting date to 2011-09 for 
Setting date to 2011 for 
Setting date to 2011 for 
Changing month from oct to 10
Setting date to 2011-10 for 
Changing month from aug to 8
Setting date to 2011-08 for 
Setting date to 2011 for 
Setting date to 2013 for 
Setting date to 2014 for 
Changing 'journal=Instrumentation and Measuremen

In [79]:
write_bib(database=bib_database, filename=BIB_FILENAME)